In [99]:
using LinearAlgebra
using Plots
using Ripserer
using Distances
using Plots
using DataFrames, CSV
using DelimitedFiles
using AbstractAlgebra
using MultivariateStats #for pca
using IterativeSolvers
using NearestNeighbors
include("mv_function_bens_alg_no_save.jl")


mv (generic function with 1 method)

In [100]:
#Globals
KKK = 2
K = 5 # in KNN
OG_K = 10
ITER = 10    # iterations of algorithm
break_index = KKK
min_loss = 1000
input_file ="./cool_big_data.csv"
output_file ="./point_color"

a_ = []
b_ = []
c_ = []
d_ = []
loss_ = []

Any[]

In [101]:
# Loss function 0.5 instead of pi

function loss(points, coord_ab, coord_cd)
    a = mod(coord_cd[points[1]],1.0)
    b = mod(coord_cd[points[2]],1.0)
    c = mod(coord_ab[points[3]],1.0)
    d = mod(coord_ab[points[4]],1.0)

    dif_1 = 0.5 - mod(a-b, 1.0)

    dif_2 = 0.5 - mod(c-d, 1.0)

    loss = (dif_1)^2 +  (dif_2)^2

    return loss
end

loss (generic function with 1 method)

In [102]:
# Load Datab
matrix = readdlm(input_file, ',', Float64)
data = []
for row in 1:length(matrix[:,1])
    arr = [matrix[row,i] for i in 1:length(matrix[1,:])]
    push!(data, arr)
end

# Do zeroth iteration of algerithm with random points
l = length(matrix[:,1])
lll = l
# Random distinct start points,
# Could be issues with distinct points ybielding same piercing hole, and therefore cant do MV
#a_0,b_0,c_0,d_0 = rand(1:25), rand(25:50), rand(50:75), rand(75:100)

a_0,b_0,c_0,d_0 = rand(1:floor(Int, l/4)), rand(floor(Int, l/4):floor(Int, l/2)), rand(floor(Int, l/2):floor(Int,3*l/4)), rand(floor(Int, 3*l/4):l)

# Store to see progress 

push!(a_, a_0)
push!(b_, b_0)
push!(c_, c_0)
push!(d_, d_0)


data_matrix = zeros(Float64, (3, length(data)))

for col in 1:length(data)
    data_matrix[:,col] = [data[col][1],data[col][2],data[col][3]]
end

kdtree = KDTree(data_matrix)
a_0,b_0,c_0,d_0

(5, 43, 63, 82)

In [103]:
#a_0,b_0,c_0,d_0 = (22, 34, 64, 81)
a_0,b_0,c_0,d_0 

(5, 43, 63, 82)

In [104]:
"""
# Perfect start
vec_0 = Array([1,0,0])
vec_1 = Array([0,1,0])
vec_2 = Array([0,0,1])

a_0_set, dists = knn(kdtree, vec_0,K, true)
c_0_set, dists = knn(kdtree, vec_1,K, true)

vec_0 = Array([-1,0,0])
vec_1 = Array([0,-1,0])
vec_2 = Array([0,0,-1])

b_0_set, dists = knn(kdtree, vec_0,K, true)
d_0_set, dists = knn(kdtree, vec_1,K, true)


push!(a_, a_0_set[1])
push!(b_, b_0_set[1])
push!(c_, c_0_set[1])
push!(d_, d_0_set[1])
"""


"# Perfect start\nvec_0 = Array([1,0,0])\nvec_1 = Array([0,1,0])\nvec_2 = Array([0,0,1])\n\na_0_set, dists = knn(kdtree, vec_0,K, true)\nc_0_set, dists = knn(kdtree, vec_1,K, true)\n\nvec_0 = Array([-1,0,0])\nvec_1 = Array([0,-1,0])\nvec_2 = Array([0,0,-1])\n\nb_0_set, dists = knn(kdtree, vec_0,K, true)\nd_0_set, dists = knn(kdtree, vec_1,K, true)\n\n\npush!(a_, a_0_set[1])\npush!(b_, b_0_set[1])\npush!(c_, c_0_set[1])\npush!(d_, d_0_set[1])\n"

In [105]:

for k in 1:ITER
    println("Current points: ")
    l = length(a_)
    println("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," * string(d_[l]-1) * "]")
    points = [a_[l], b_[l], c_[l], d_[l]]


    bool_boy = true

    a_0 = points[1]
    b_0 = points[2]
    d_0 = points[4]
    c_0 = points[3]

    vec_0 = Array([data[a_0][1],data[a_0][2],data[a_0][3]])
    vec_1 = Array([data[b_0][1],data[b_0][2],data[b_0][3]])
    vec_2 = Array([data[c_0][1],data[c_0][2],data[c_0][3]])
    vec_3 = Array([data[d_0][1],data[d_0][2],data[d_0][3]])
    
    
    a_0_set, dists = knn(kdtree, vec_0,K, true)
    b_0_set, dists = knn(kdtree, vec_1,K, true)
    c_0_set, dists = knn(kdtree, vec_2,K, true)
    d_0_set, dists = knn(kdtree, vec_3,K, true)
    


    # find b_0^i and d_0^i that minimizes loss
    a_0_i = a_0
    b_0_i = b_0
    c_0_i = c_0
    d_0_i = d_0

    # See if stationary
    old_a = a_0
    old_b = b_0
    old_c = c_0
    old_d = d_0
    
    for i in 1:length(b_0_set)
        b_0_i = b_0_set[i]
        
        coord_ab, coord_cd = mv(points = [a_0,b_0_i, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        l = loss([a_0,b_0_i,c_0,d_0], coord_ab, coord_cd)
        if  l < min_loss
            min_loss = l
            b_0 = b_0_i

        end
  

    end
    
    for i in 1:length(c_0_set)
        c_0_i = c_0_set[i]
        coord_ab, coord_cd = mv(points = [a_0,b_0, c_0_i, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        l = loss([a_0,b_0,c_0_i,d_0], coord_ab, coord_cd)
        if  l < min_loss
            min_loss = l
            c_0 = c_0_i
            bool_boy = false
        end
      
    end
    if bool_boy
        for i in 1:length(d_0_set)
            d_0_i = d_0_set[i]
            coord_ab, coord_cd = mv(points = [a_0,b_0, c_0, d_0_i], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
            l = loss([a_0,b_0,c_0,d_0_i], coord_ab, coord_cd)
            if  l < min_loss
                min_loss = l
                d_0 = d_0_i
            end
        end
    end
    """
    try 
        coord_ab, coord_cd
    catch
        l = lll
        println("Ok, points suck, try new points",  [a_0,b_0,c_0,d_0] )
        a_0,b_0,c_0,d_0= rand(1:floor(Int, l/4)), rand(floor(Int, l/4):floor(Int, l/2)), rand(floor(Int, l/2):floor(Int,3*l/4)), rand(floor(Int, 3*l/4):l)
        coord_ab, coord_cd = mv(points = [a_0,b_0, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
    end
    """

    if old_b == b_0 && old_d == d_0 && old_a == a_0 && old_c == c_0
        break_index -=1
        println("THIS IS A FIXPOINT")
    else
        break_index = KKK
    end

    println("")
    println("Loss: ", min_loss)
    push!(loss_, min_loss)

    println("")
    push!(a_, a_0)
    push!(b_, b_0)
    push!(c_, c_0)
    push!(d_, d_0)

    if break_index == 0
        #mv(points = [a_0,b_0_i, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        break
    end
    println("________________________________________________________________")
    println("Done with: ", k, " Break_index: ", break_index)
    println("________________________________________________________________")

end

println("DONE")
mv(points = [a_0,b_0, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
println("DONE")



Current points: 
[4,42,62,81]



Loss: 0.3401799957141878

________________________________________________________________
Done with: 1 Break_index: 2
________________________________________________________________
Current points: 
[4,52,56,81]



Loss: 0.2681549683803055

________________________________________________________________
Done with: 2 Break_index: 2
________________________________________________________________
Current points: 
[4,28,69,81]



Loss: 0.12962343396827866

________________________________________________________________
Done with: 3 Break_index: 2
________________________________________________________________
Current points: 
[4,46,34,81]



Loss: 0.07450029902735443

________________________________________________________________
Done with: 4 Break_index: 2
________________________________________________________________
Current points: 
[4,10,33,81]



Loss: 0.04461077429152074

________________________________________________________________
Done with: 5 Break_index: 2
________________________________________________________________
Current points: 
[4,59,60,81]



Loss: 0.012340180995503616

________________________________________________________________
Done with: 6 Break_index: 2
________________________________________________________________
Current points: 
[4,59,60,12]



Loss: 0.0013212098299891098

________________________________________________________________
Done with: 7 Break_index: 2
________________________________________________________________
Current points: 
[4,59,60,26]


THIS IS A FIXPOINT

Loss: 0.0013212098299891098

________________________________________________________________
Done with: 8 Break_index: 1
________________________________________________________________
Current points: 
[4,59,60,26]


THIS IS A FIXPOINT

Loss: 0.0013212098299891098

DONE
DONE

In [106]:
loss_

9-element Vector{Any}:
 0.3401799957141878
 0.2681549683803055
 0.12962343396827866
 0.07450029902735443
 0.04461077429152074
 0.012340180995503616
 0.0013212098299891098
 0.0013212098299891098
 0.0013212098299891098

In [107]:
println("Input:")

println("[" * string(a_[1]-1) * "," * string(b_[1]-1) * "," * string(c_[1]-1) * "," * string(d_[1]-1) * "]")

Input:
[4,42,62,81]


In [108]:
l = length(a_)
println("Output:")
print("[" * string(a_[l]-1) * "," * string(b_[l]-1) * "," * string(c_[l]-1) * "," * string(d_[l]-1) * "]")

Output:
[4,59,60,26]

In [109]:
"""
for i in 1:length(b_0_set)
    b_0_i = b_0_set[i]
    try
        coord_ab, coord_cd = mv(points = [a_0,b_0_i, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        l = loss([a_0,b_0_i,c_0,d_0], coord_ab, coord_cd)
        if  l < min_loss
            min_loss = l
            b_0 = b_0_i

        end
    catch
        println("Starting points suck = ", [a_0,b_0_i, c_0, d_0], "   ", b_0_i)
        breal
        
    end     




end


for i in 1:length(c_0_set)
    c_0_i = c_0_set[i]
    try
        coord_ab, coord_cd = mv(points = [a_0,b_0, c_0_i, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        l = loss([a_0,b_0,c_0_i,d_0], coord_ab, coord_cd)
        if  l < min_loss
            min_loss = l
            c_0 = c_0_i

        end
    catch
        println("Starting points suck = ", [a_0,b_0, c_0_i, d_0], "   ", c_0_i)
        break
        
    end        
end

for i in 1:length(d_0_set)
    d_0_i = d_0_set[i]
    try
        coord_ab, coord_cd = mv(points = [a_0,b_0, c_0, d_0_i], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
        l = loss([a_0,b_0,c_0,d_0_i], coord_ab, coord_cd)
        if  l < min_loss
            min_loss = l
            d_0 = d_0_i
        end
    catch
        println("Starting points suck = ", [a_0,b_0, c_0, d_0_i], "   ", d_0_i)
        break
    end     
    

end

try 
    coord_ab, coord_cd
catch
    l = lll
    println("Ok, points suck, try new points",  [a_0,b_0,c_0,d_0] )
    a_0,b_0,c_0,d_0= rand(1:floor(Int, l/4)), rand(floor(Int, l/4):floor(Int, l/2)), rand(floor(Int, l/2):floor(Int,3*l/4)), rand(floor(Int, 3*l/4):l)
    coord_ab, coord_cd = mv(points = [a_0,b_0, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)
end
"""

"for i in 1:length(b_0_set)\n    b_0_i = b_0_set[i]\n    try\n        coord_ab, coord_cd = mv(points = [a_0,b_0_i, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)\n        l = loss([a_0,b_0_i,c_0,d_0], coord_ab, " ⋯ 1385 bytes ⋯ "), rand(floor(Int, l/4):floor(Int, l/2)), rand(floor(Int, l/2):floor(Int,3*l/4)), rand(floor(Int, 3*l/4):l)\n    coord_ab, coord_cd = mv(points = [a_0,b_0, c_0, d_0], start_points = true, input_file = input_file, output_file = output_file, double_penetration = true)\nend\n"